In [28]:
from IPython.display import display, Image
import cv2 as cv
from PIL import Image

In [17]:
#display(Image(filename='brad.jpg'))
#display(Image(filename='kim.jpg'))
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier("haarcascade_eye.xml")

In [32]:
img = cv.imread('kim.jpg')
im = Image.open("brad.jpg") 

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray)

for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    roi_gray = gray[y : y + h, x : x + h]
    roi_color = img[y : y + h, x : x + h]
    # Otsi silmi vaid leitud nägude aladelt (kollased ristkülikud)
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)
    im1 = im.crop((x, y, x+w, y+h))
    im1.show()

# Otsi silmi kogu fotolt (rohelised ristkülikud)
eyes = eye_cascade.detectMultiScale(gray)
for (ex, ey, ew, eh) in eyes:
    cv.rectangle(img, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    

cv.imshow('Pealkiri', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [27]:
# Pildi salvestamiseks
#cv.imwrite('jott_koopia.png', img)